<h1><span style="color:purple"><strong>SOURCE EXTRACTOR for PYTHON (SEP)</strong></span></h1>

<h3><span style="color:orange">PHOTOGRAPHIC PLATE USE CASE

---

<span style="color:olive"><strong>SETUP

In [ ]:
# install sep with pip, may need to run command in terminal then restart notebook
# https://sep.readthedocs.io/en/v1.1.x/index.html#installation

pip install sep

In [ ]:
# import libraries

import sep
import numpy as np
import os
import shutil
import pandas as pd

from astropy import wcs
from astropy.io import fits

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

In [ ]:
# change file and directory, file should have WCS solution embedded
directory = "directory/to/file/"
file = "file.fits"

# choose whether or not to plot results (True will result in longer time!)
plot = True
# choose whether to save both the background subtracted image and catalog
save = True

# define an appropriate transformation function
def transform(data):
    '''
    outputs the transformed data, in the same image shape
    note that sep only works with np.float64 or np.int16
    '''
    return 65536.-data.astype(np.float64)
    # return data.astype(np.float64)

<span style="color:olive"><strong>EXTRACT OBJECTS</strong>

<span style="color:olive">calculate and subtract background

In [ ]:
# opens given FITS file
fits_file = fits.open(os.path.join(directory,file))
header,data = fits_file[0].header,fits_file[0].data

# transforms the data
data_t = transform(data)

# https://sep.readthedocs.io/en/v1.1.x/api/sep.Background.html
# find and subtract background
# all inputs are currently set to defaults
#   mask is a boolean mask array, same size as data
#   bw,bh are size of background boxes in pixels
#   fw,fh are filter width and height in boxes
#   fthresh is the filter threshold
bkg = sep.Background(data=data_t,
                     mask=None,
                     bw=64,
                     bh=64,
                     fw=3,
                     fh=3,
                     fthresh=0)
bkg_rms = bkg.rms()
sdata = data_t-bkg

if save:
    fits.writeto(os.path.join(directory,file[:-5]+'_bksub.fits'),sdata,header,overwrite=True)
    print("background subtracted/transformed data saved as:",file[:-5]+'_bksub.fits')

if plot:
    plt.figure(figsize=(50,10)).set_facecolor('lightgrey')
    plt.subplot(1,5,1)
    plt.title("ORIGINAL DATA")
    plt.imshow(data,cmap='magma',origin="lower")
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.subplot(1,5,2)
    plt.title("TRANSFORMED DATA")
    plt.imshow(data_t,cmap='magma',origin="lower")
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.subplot(1,5,3)
    plt.title("BACKGROUND")
    plt.imshow(bkg,cmap='magma',origin="lower")
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.subplot(1,5,4)
    plt.title("BACKGROUND RMS")
    plt.imshow(bkg_rms,cmap='magma',origin="lower")
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.subplot(1,5,5)
    plt.title("BACKGROUND SUBTRACTED/TRANSFORMED DATA")
    plt.imshow(sdata,cmap='magma',origin="lower")
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.show()

<span style="color:olive">extract objects

In [ ]:
# https://sep.readthedocs.io/en/v1.1.x/api/sep.extract.html
# find objects in background subtracted transformed image
# will mention a few important parameters, others are described in docs
#   thresh is a relative threshold from the background error
#   mask is a boolean mask array, same size as data
#   min_area gives the minimum amount of pixels required to count as an object
#   clean removes objects that are unlikely to be real

objects,segmap = sep.extract(data=sdata,
                             thresh=3,
                             err=bkg.globalrms,
                             mask=None,
                             maskthresh=0,
                             minarea=10,
                             filter_kernel=np.array([[1,2,1],
                                                     [2,4,2],
                                                     [1,2,1]]),
                             filter_type="matched",
                             deblend_nthresh=32,
                             deblend_cont=0.005,
                             clean=False,
                             clean_param=1.0,
                             segmentation_map=True)
df = pd.DataFrame(objects)
print(f'a total of {len(objects)} objects found')

if plot:
    # plot background-subtracted image
    plt.figure(figsize=(40,40)).set_facecolor('lightgrey')
    plt.title("OBJECTS")
    ax = plt.gca()
    plt.imshow(sdata,cmap='magma',origin='lower',interpolation='none')
    # plot an ellipse for each object
    # based on: https://sep.readthedocs.io/en/v1.1.x/tutorial.html
    for i in range(len(objects)):
        e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                    width=6*objects['a'][i],
                    height=6*objects['b'][i],
                    angle=objects['theta'][i] * 180. / np.pi)
        e.set_facecolor('none')
        e.set_edgecolor('turquoise')
        ax.add_artist(e)
    plt.show()

    # plot segmentation map
    # plt.figure(figsize=(40,40)).set_facecolor("lightgrey")
    # plt.imshow(segmap,cmap="Spectral",origin="lower",interpolation="none")

In [ ]:
# calculate RAs and Decs using WCS solution
# uses member pixel's barycenter for each object
w = wcs.WCS(header)
ras,decs = w.all_pix2world(df["x"],df["y"],0)
df["ra"],df["dec"] = ras,decs


# also get RAs and Decs using windowed algorithm
# https://sep.readthedocs.io/en/v1.1.x/api/sep.winpos.html
x_,y_,flag_ = sep.winpos(data=sdata,
                         xinit=df['x'],
                         yinit=df['y'],
                         sig=2)
df["x_win"],df["y_win"] = x_,y_
ras_,decs_ = w.all_pix2world(df["x_win"],df["y_win"],0)
df["ra_win"],df["dec_win"] = ras_,decs_

# finally, get RAs and Decs using the maximum flux point
# using convolved, to avoid random local maxima due to noise
ras__,decs__ = w.all_pix2world(df["xcpeak"],df["ycpeak"],0)
df["ra_flux"],df["dec_flux"] = ras__,decs__



# saves object catalog
if save:
    df.to_csv(os.path.join(directory,file[:-5]+'_obj.csv'))
    print("Source Extractor Objects saved in "+file[:-5]+'_obj.csv')

In [ ]:
# save segmap
if save:
    fits.writeto(os.path.join(directory,file[:-5]+'_segmap.fits'),segmap,header,overwrite=True)

---

<span style="color:olive"><strong>CATALOGS</strong>

In [ ]:
# calculate a circle that encompasses the entire plate's area
# w = wcs.WCS(header)

cra,cdec = w.all_pix2world(header["NAXIS1"]/2,header["NAXIS2"]/2,0)
radius = (((w.all_pix2world(header["NAXIS1"],header["NAXIS2"],0)[0]-w.all_pix2world(1,1,0)[0])**2+
          (w.all_pix2world(header["NAXIS1"],header["NAXIS2"],0)[1]-w.all_pix2world(1,1,0)[1])**2)**0.5)/2
print(f"search within {radius:.3f}° from {cra:.3f},{cdec:.3f}")

<span style="color:olive">Gaia

In [ ]:
from astroquery.gaia import Gaia

# change YEAR in .format line to year that plate was taken
# currently, this precesses astrometry of sources within a circular field covering the plate area
# returns other infomration, like magnitudes and flags
# there is a magntiude cut, but this can be removed

job = Gaia.launch_job_async(
    """
    SELECT 
        source_id,ra as ra_J2016,array_element(a0,1) as ra_J{0},dec as dec_J2016,array_element(a0,2) as dec_J{0},
        parallax as parallax_J2016,array_element(a0,3) as parallax_J{0},parallax_error,
        pmra as pmra_J2016,array_element(a0,4) as pmra_J{0},pmra_error,
        pmdec as pmdec_J2016,array_element(a0,5) as pmdec_J{0},pmdec_error,
        radial_velocity as radial_velocity_J2016,array_element(a0,6) as radial_velocity_J{0},
        phot_g_mean_mag,phot_bp_mean_mag,phot_bp_mean_flux_over_error,phot_rp_mean_mag,phot_rp_mean_flux_over_error,
        bp_rp,phot_variable_flag,classprob_dsc_combmod_galaxy
    FROM
       (
        SELECT TOP 500000 
            source_id,ra,dec,parallax,parallax_error,pmra,pmra_error,pmdec,pmdec_error,radial_velocity,
            phot_g_mean_mag,phot_bp_mean_mag,phot_bp_mean_flux_over_error,phot_rp_mean_mag,phot_rp_mean_flux_over_error,
            bp_rp,phot_variable_flag,classprob_dsc_combmod_galaxy,
            epoch_prop(ra,dec,parallax,pmra,pmdec,radial_velocity,2016.0,{0}) as a0
        FROM gaiadr3.gaia_source
        WHERE CONTAINS(POINT('ICRS',ra,dec),CIRCLE('ICRS',{1},{2},{3}))=1 AND ((phot_bp_mean_mag + 1.1*bp_rp) < 21.5)
        ) as p
    """.format(YEAR,cra,cdec,radius),
    dump_to_file=True, output_format='csv')

# save and rename the output file in the correct place
os.replace(job.outputFile,file[:-5]+'_gaia.csv')
shutil.move(file[:-5]+'_gaia.csv',os.path.join(directory,file[:-5]+'_gaia.csv'))
gaia_data = pd.read_csv(os.path.join(directory,file[:-5]+'_gaia.csv'))
print(str(len(gaia_data))+" objects detected in field")
print("\nGaia Objects saved as "+file[:-5]+'_gaia.csv')

<span style="color:olive">SDSS

In [ ]:
from astroquery.sdss import SDSS

# catalog; for example, star or galaxy
catalog = "star"

# run SDSS query in circle
query = """
        SELECT TOP 500000
            c.objID,c.ra,c.dec,c.u,c.err_u,c.g,c.err_g,c.r,c.err_r,c.i,c.err_i,c.z,c.err_z,s.z as redshift,c.clean
        FROM {0} c
        JOIN dbo.fGetNearbyObjEq({1},{2},{3}) n ON c.objID=n.objID
        LEFT JOIN SpecObj s ON c.objID=s.bestObjID
        WHERE g<20.5
        """.format(catalog,cra,cdec,radius*60)
res = SDSS.query_sql(query,data_release=17)

# save output file
print(str(len(res))+" "+catalog+"s detected in field")
res.write(os.path.join(directory,file[:-5]+"_sdss-"+catalog+".csv"),overwrite=True)
print("\nSDSS Objects saved as "+file[:-5]+"_sdss-"+catalog+".csv")

---

<span style="color:olive"><strong>PHOTOMETRY

<span style="color:olive">circle aperture

In [ ]:
# https://sep.readthedocs.io/en/v1.1.x/api/sep.sum_circle.html
for r in np.arange(2.5,9.5,0.5):
    flux0,_,_ = sep.sum_circle(data=sdata, 
                                      x=df['x'], 
                                      y=df['y'], 
                                      r=r)
    df["flux_bary_"+str(r)] = flux0

In [ ]:
df.to_csv(os.path.join(directory,file[:-5]+'_obj.csv'))

In [ ]:
plt.figure(figsize=(10,4),dpi=200).set_facecolor("lightgray")
plt.title("COMPARISON")
_,bins,_ = plt.hist(flux0,alpha=0.5,bins=np.linspace(0,3e6,100),color="red",label="circle flux")
plt.hist(df['flux'],alpha=0.5,bins=bins,color="gray",label="member flux")
plt.legend()
plt.show()

<span style="color:olive">circle annular aperture

In [ ]:
# https://sep.readthedocs.io/en/v1.1.x/api/sep.sum_circann.html
flux1,fluxerr1,flag1 = sep.sum_circann(data=sdata, 
                                       x=df['x'], 
                                       y=df['y'], 
                                       rin=0.,
                                       rout=5.)

In [ ]:
plt.figure(figsize=(10,4),dpi=200).set_facecolor("lightgray")
plt.title("COMPARISON")
_,bins,_ = plt.hist(flux1,alpha=0.5,bins=np.linspace(0,3e6,100),color="orange",label="circle ann flux")
plt.hist(df['flux'],alpha=0.5,bins=bins,color="gray",label="member flux")
plt.legend()
plt.show()

<span style="color:olive">ellipse aperture

In [ ]:
# https://sep.readthedocs.io/en/v1.1.x/api/sep.sum_ellipse.html
flux2,fluxerr2,flag2 = sep.sum_ellipse(data=sdata, 
                                       x=df['x'], 
                                       y=df['y'], 
                                       a=df['a'], 
                                       b=df['b'], 
                                       theta=df['theta'], 
                                       r=1.)

In [ ]:
plt.figure(figsize=(10,4),dpi=200).set_facecolor("lightgray")
plt.title("COMPARISON")
_,bins,_ = plt.hist(flux2,alpha=0.5,bins=np.linspace(0,3e6,100),color="blue",label="ellipse flux")
plt.hist(df['flux'],alpha=0.5,bins=bins,color="gray",label="member flux")
plt.legend()
plt.show()

<span style="color:olive">ellipse annular aperture

In [ ]:
# https://sep.readthedocs.io/en/v1.1.x/api/sep.sum_ellipann.html
flux3,fluxerr3,flag3 = sep.sum_ellipann(data=sdata, 
                                        x=df['x'], 
                                        y=df['y'], 
                                        a=df['a'], 
                                        b=df['b'], 
                                        theta=df['theta'], 
                                        rin=0.,
                                        rout=1.)

In [ ]:
plt.figure(figsize=(10,4),dpi=200).set_facecolor("lightgray")
plt.title("COMPARISON")
_,bins,_ = plt.hist(flux3,alpha=0.5,bins=np.linspace(0,3e6,100),color="purple",label="ellipse ann flux")
plt.hist(df['flux'],alpha=0.5,bins=bins,color="gray",label="member flux")
plt.legend()
plt.show()

<span style="color:olive">Kron radius

In [ ]:
# https://sep.readthedocs.io/en/v1.1.x/api/sep.kron_radius.html
kronrad4,flag4 = sep.kron_radius(data=sdata, 
                                 x=df['x'], 
                                 y=df['y'], 
                                 a=df['a'], 
                                 b=df['b'], 
                                 theta=df['theta'],  
                                 r=6.)

<span style="color:olive">flux radius

In [ ]:
# https://sep.readthedocs.io/en/v1.1.x/api/sep.flux_radius.html
radius5,flag5 = sep.flux_radius(data=sdata, 
                                x=df['x'], 
                                y=df['y'],
                                rmax=6.*df['a'],
                                frac=0.5,
                                normflux=df['flux'])

---